In [ ]:
from pathlib import Path
from typing import List

import functools
import operator 

import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable


from kedro.extras.datasets.pickle import PickleDataSet
from kedro.config import ConfigLoader

import kornia.augmentation as K

import monai
from monai.networks import one_hot
from scipy import ndimage

from skimage.draw import disk

import torch
from torch.utils.data import DataLoader, TensorDataset, random_split
from torchvision.utils import save_image

In [ ]:
import os, sys
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../src/'))

from src.tagseg.metrics.shape import ShapeLoss

In [ ]:
def losses(input, target, smooth_k):
    input = torch.Tensor(input).float().unsqueeze(0)
    target = torch.Tensor(target).long().unsqueeze(0).unsqueeze(0)

    sl = ShapeLoss(to_onehot_y=True, include_background=False, smooth_k=smooth_k)(input, target)
    cel = torch.nn.CrossEntropyLoss()(input, target[0])
    hd = monai.metrics.compute_hausdorff_distance(input[0, 1].unsqueeze(0).unsqueeze(0), target)

    return sl.item(), cel.item(), hd.item()

In [ ]:
def distance_maps(target, smooth_k=1e-2):

        target = torch.Tensor(target).long().unsqueeze(0).unsqueeze(0)

        target = one_hot(target, 2)
        target = target[:, 1:]

        distance_maps = torch.Tensor()

        for _im in range(target.shape[0]):
            distance_map = torch.Tensor()

            for _class in range(target.shape[1]):
                roi = target[_im, _class].numpy()

                dt = ndimage.distance_transform_edt(roi)
                dt /= dt.max() + 1e-8
                dt_n = ndimage.distance_transform_edt(1 - roi)
                dt_n /= dt_n.max() + 1e-8

                shape_information = (1 - dt) #  * roi + (dt_n - 1) * (1 - roi)
                sdm = torch.Tensor(1 / (1 + np.exp(-shape_information / smooth_k))) * roi
                distance_map = torch.cat([distance_map, sdm.unsqueeze(0)], axis=0)

            distance_maps = torch.cat([distance_maps, distance_map.unsqueeze(0)], axis=0)

        distance_maps = distance_maps.to(target.device)

        return distance_maps

In [ ]:
cases = []

input = np.zeros((256, 256))
input[disk((128, 128), 64)] = 1
input[disk((128, 128), 32)] = 0
input = np.array([1 - input, input])

target = np.zeros((256, 256))
target[disk((128, 128), 64)] = 1
target[disk((128, 128), 32)] = 0

cases.append(dict(input=input, target=target))

input = np.zeros((256, 256))
input[disk((128, 128), 86)] = 1
input[disk((128, 128), 16)] = 0
input = np.array([1 - input, input])

target = np.zeros((256, 256))
target[disk((128, 128), 64)] = 1
target[disk((128, 128), 32)] = 0

cases.append(dict(input=input, target=target))

input = np.zeros((256, 256))
input[disk((128, 128), 64)] = 1
input[disk((128, 128), 32)] = 0
input = np.array([1 - input, input])

target = np.zeros((256, 256))
target[disk((128, 128), 86)] = 1
target[disk((128, 128), 16)] = 0

cases.append(dict(input=input, target=target))

input = np.zeros((256, 256))
input[disk((180, 180), 64)] = 1
input[disk((180, 180), 32)] = 0
input = np.array([1 - input, input])

target = np.zeros((256, 256))
target[disk((90, 90), 64)] = 1
target[disk((90, 90), 32)] = 0

cases.append(dict(input=input, target=target))

input = np.zeros((256, 256))
input[disk((132, 128), 64)] = 1
input[disk((132, 128), 32)] = 0
input = np.array([1 - input, input])

target = np.zeros((256, 256))
target[disk((112, 128), 64)] = 1
target[disk((112, 128), 32)] = 0

cases.append(dict(input=input, target=target))

In [ ]:
plt.rcParams['ytick.right'] = plt.rcParams['ytick.labelright'] = True
plt.rcParams['ytick.left'] = plt.rcParams['ytick.labelleft'] = False

fig, ax = plt.subplots(4, len(cases), figsize=(6, 6))

for i, case in enumerate(cases):

    ax[0, i].imshow(case['input'][1], cmap='Blues', alpha=0.5)
    ax[0, i].imshow(case['target'], cmap='Reds', alpha=0.5)
    ax[0, i].axis('off')


    dm = distance_maps(case['target'], smooth_k=.2)
    im = ax[1, i].imshow(dm[0, 0], cmap='cividis', vmin=0., vmax=1.)
    ax[1, i].axis('off')

    if i == len(cases) - 1:
        divider = make_axes_locatable(ax[1, i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax)

    diff = (torch.Tensor(case['input']).unsqueeze(0)[:, 1:] - dm).abs()
    im = ax[2, i].imshow(diff[0, 0], cmap='cividis', vmin=0., vmax=1.)
    ax[2, i].axis('off')

    if i == len(cases) - 1:
        divider = make_axes_locatable(ax[2, i])
        cax = divider.append_axes('right', size='5%', pad=0.05)
        plt.colorbar(im, cax=cax)

    loss_names = ['SL', 'CEL', 'HD']
    loss_values = list(losses(case['input'], case['target'], smooth_k=.2))
    ax[3, i].bar(loss_names[:2], loss_values[:2], color=['#219EBC', '#FFB703'])
    ax[3, i].set_ylim(0, 2.0)
    if i < len(cases) - 1:
        ax[3, i].set_yticks([])

plt.tight_layout()

In [ ]:
fig.savefig('../figures/shape-loss.svg', dpi=150, format='svg')